# Preprocessing, GridSearch and Training SpiderNet-8 on Private Data without B/W-tests features

In [1]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
import re
import gc

import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
task = 'binary'
missing_strings = 'MISSING'
p_value = 0.05
target_dict = {'good': 0, 'bad': 1}

In [7]:
target = 'BAD_FLAG'
index_month = 'MONTH_YEAR'
list_of_vars_for_strat = ['MONTH_YEAR']
sort_by_var = 'APPPOSID'

necessary_fields = [target, index_month, sort_by_var]

COL_DEL = ['Unnamed: 0', 'PERIOD_7', 'LOSS_90P'] 
COL_DEL = [x.upper() for x in COL_DEL]
COL_TRG = target

In [8]:
COL_DEL

['UNNAMED: 0', 'PERIOD_7', 'LOSS_90P']

In [9]:
necessary_fields

['BAD_FLAG', 'MONTH_YEAR', 'APPPOSID']

In [10]:
n_jobs = 3

## Start of selection

In [11]:
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Dropout)
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, AveragePooling1D)
from tensorflow.keras.layers import add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

In [12]:
from tensorflow.keras.layers import ReLU, Concatenate, GlobalAveragePooling1D

In [13]:
from keras.utils import np_utils
from tensorflow.keras.models import Model

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, Callback, LearningRateScheduler
#from keras.callbacks import EarlyStopping, Callback

# объявляем класс метрик

class E_time(Callback):
    def on_train_begin(self, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('train_begin', '| time: ' , tm)
        
    def on_epoch_end(self, epoch, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('epoch_end', '| time: ' , tm)
    
        return

_time = E_time()

In [15]:
import tensorflow as tf

In [16]:
print(tf.__version__)

2.3.0


In [17]:
#если alpha не задано, то не делаем br_correction

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        #return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)
        return [np.array(X)], np.array(y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping

## Loading Data

In [19]:
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/'
#загружаем данные для 163 переменных

train_for = pd.read_csv(PATH + 'train_163_prep.csv')
valid_for = pd.read_csv(PATH + 'valid_163_prep.csv')
test_for = pd.read_csv(PATH + 'test_163_prep.csv')

In [20]:
y_train = train_for[target]
#y_test = valid_for[target]
#y_val = test_for[target]

y_test = test_for[target]
y_val = valid_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2 = train_for[col]
X_2_2 = valid_for[col]
X_3_2 = test_for[col]

In [21]:
#проверка корректности
X_1_2.shape, train_for.shape, len(y_train)

((1507599, 163), (1507599, 164), 1507599)

In [22]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [23]:
#проверка корректности, должно быть везде True
print(X_2_2.shape[0] == valid_for.shape[0])
print(X_2_2.shape[1] == valid_for.shape[1] - 1)
print(len(y_val) == valid_for.shape[0])

True
True
True


In [24]:
#если alpha не задано, то не делаем br_correction
#новая версия, исправлены индексы

class DataGenerator_new(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in indexes]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [25]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
#validation_generator = DataGenerator(valid_for, x_col, y_col, batch_size=valid_for.shape[0], alpha = None, class_w = None)

In [26]:
Y_test_2 = np_utils.to_categorical( y_val, 2) # преобразовываем в 2 класса 

In [27]:
X_2_2.shape, Y_test_2.shape

((185400, 163), (185400, 2))

# *

In [28]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 29

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)


# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# ***

In [29]:
from tensorflow.keras.layers import GlobalMaxPooling1D

In [30]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = self.model.predict(self.validation_data[0])[:, 1]
        val_targ = self.validation_data[1]
        
        if len(val_targ.shape) == 2: #and val_targ.shape[1] != 1:
            val_targ = val_targ[:,1]

        _val_aps = metrics.average_precision_score(val_targ, val_predict)
        #_val_recall = metrics.recall_score(val_targ, val_predict)
        _val_a = metrics.roc_auc_score(val_targ, val_predict)

        logs['val_aps'] = _val_aps
        logs['val_a'] = _val_a
        print(" — val_aps:  %f — val_a: %f" % (_val_aps, _val_a))
        return

In [31]:
import os
os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\'

In [32]:
#training_aug = DataGenerator(aug_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )

In [33]:
#learning_rate=0.005
initial_learning_rate = 0.005

def lr_step_decay(epoch, lr):
    drop_rate = 0.1
    epochs_drop = 20.0
    
    return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

In [34]:
import urllib
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

In [35]:
from tensorflow.keras.models import load_model

## GRID SEARCH

## SpiderNet-8

In [36]:
def max_pool(min_pool, n_pool, padding_pool, str_pool, y ):
    if y.shape[1] < (min_pool):
        
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=1)(y)
    else:
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=str_pool)(y)

In [37]:
def spider_model_8block(l2_conv = None, reg = 1E-5 , reg_dense = 1E-5,
                 _bias = True,  n_features = 163, n_pool = 2, n_kernel = 5, n_filters = 5, n_strides = 1,
                 classes = 2, hidden = 64 , drop_out = 0.25, drop_out_conv = 0.001, drop_out_rate = 4 ,
                 padding_pool = 'valid' ,
                gl_pool_max = False,
                       min_pool = 12, str_pool = None):
    
    if reg == None:
        l2_batch_gamma = None 
        l2_batch_betta = None
    else:
        l2_batch_gamma = l2(reg)
        l2_batch_betta =l2(reg)
    
    if reg_dense == None:
        kernel_regularizer = None
    else:
        kernel_regularizer = l2(reg)   
    
    
    x = Input(shape=(  n_features, 1))
    n = -2 # для 6 - n = 0, для 8 = -2
    y = Conv1D(filters=n_filters, kernel_size=n_kernel, strides=n_strides, padding='same', 
           use_bias=_bias, kernel_regularizer=l2_conv)(x) 

    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
#y = Dropout(rate=drop_rate)(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut1_2 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)## поправить
    shortcut1_3 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_2)##
    shortcut1_4 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_3)##
    
    shortcut1_5 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_4)
    shortcut1_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_5) #*
    shortcut1_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_6) #*

    # второй spider-block
    y = Conv1D(filters=n_filters*2, kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut2_3 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)##
    shortcut2_4 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut2_3)##
    
    shortcut2_5 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut2_4)
    shortcut2_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut2_5)
    shortcut2_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut2_6)


    y= Concatenate(axis=-1)([shortcut1_2, y])
    n = n+1

    # третий spider-block
    y = Conv1D(filters=n_filters*(3 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut3_4 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)##
    
    shortcut3_5 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut3_4)##
    shortcut3_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut3_5)##
    shortcut3_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut3_6)##

    y= Concatenate(axis=-1)([shortcut2_3, shortcut1_3, y])
    
    n = n+1
    # четвертый spider-block
    y = Conv1D(filters=n_filters*(4 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut4_5 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)##
    shortcut4_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut4_5)##
    shortcut4_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut4_6)##

    y= Concatenate(axis=-1)([shortcut3_4, shortcut2_4, shortcut1_4, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))

    # пятый spider-block
    y = Conv1D(filters=n_filters*(5 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut5_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)
    shortcut5_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut5_6)    
    
    
    y= Concatenate(axis=-1)([shortcut4_5, shortcut3_5, shortcut2_5, shortcut1_5, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))

    # шестой spider-block
    y = Conv1D(filters=n_filters*(6 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    
    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut6_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)
    
    y= Concatenate(axis=-1)([shortcut5_6, shortcut4_6, shortcut3_6, shortcut2_6, shortcut1_6, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))
    
    
    # седьмой блок
    y = Conv1D(filters=n_filters*(7 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    
    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    
    y= Concatenate(axis=-1)([shortcut6_7, shortcut5_7, shortcut4_7, shortcut3_7, shortcut2_7, shortcut1_7, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))
    
     # восьмой блок
    y = Conv1D(filters=n_filters*(8 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    
    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    
    if gl_pool_max:
        z = GlobalMaxPooling1D()(y)
        
    else:
        z = GlobalAveragePooling1D()(y)
    #z = Flatten()(y) #сглаживание, пример использования - https://stackoverflow.com/questions/43237124/what-is-the-role-of-flatten-in-keras
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    predictions = Dense(classes, activation='softmax')(z)

    #model = Sequential()
    model_15 = Model(inputs=x, outputs=predictions)
    
    return model_15

In [38]:
# for 124 features
list_train_for = pd.read_csv('124+target_wo_btest.csv', sep = ',')
col_train_all = list_train_for['var_wo_btest'].to_list()
col_train = list_train_for['var_wo_btest'].to_list()
col_train.remove(target)
len(col_train)

124

In [39]:
X_1_2 = train_for[col_train]
X_2_2 = valid_for[col_train]
X_3_2 = test_for[col_train]

In [40]:
train_for = train_for[col_train_all]

In [41]:
#model_spider8_internal_grid_20_7_5_64_0.0002_200515_2.h5


In [42]:
gl_p = [False] #0
l2_batch = [ 0.0002] #1
n_ker = [7] #2
n_fil = [ 5] #3
d_hidden = [64] #4
drop_out = [0.25] #5
drop_out_conv = [0.001] #6
reg_dense = [ 0.0002] #7
min_pool = [ 20] #7

In [43]:
param = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, drop_out_conv, reg_dense, min_pool)

In [44]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [45]:
#training_aug = DataGenerator(aug_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )

In [46]:
inp_shape = X_2_2.shape[1]
inp_shape

124

In [47]:
len(x_col)

124

In [48]:
for i in param:
    print(i)

(False, 0.0002, 7, 5, 64, 0.25, 0.001, 0.0002, 20)


In [49]:
param = [(False, 0.0002, 7, 5, 64, 0.25, 0.001, 0.0002, 20)]
param

[(False, 0.0002, 7, 5, 64, 0.25, 0.001, 0.0002, 20)]

In [50]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0:0.05 , 1:1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)
    
    model_grid = spider_model_8block(reg = p[1], reg_dense = p[7], n_features = inp_shape, n_pool = 2, n_kernel = p[2], 
                              n_filters = p[3], n_strides = 1, classes = 2, 
                              hidden = p[4] , drop_out = p[5], drop_out_conv = p[6], drop_out_rate = 4 ,
                 padding_pool = 'valid' , gl_pool_max = p[0], min_pool = p[8], str_pool = None)
    
    model_grid.compile(loss='categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    
    history_XX = model_grid.fit_generator(generator=training_generator,  #training_aug,
                    validation_data= (X_2_2, Y_test_2) , #Y_test_2 = np_utils.to_categorical( y_val, 2) 
                                     epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                     callbacks=[_time , EarlyStopping(monitor='val_loss', patience=20),
                                               Metrics(valid_data=(X_2_2, Y_test_2))])
    
    res_model_ = pd.DataFrame(history_XX.history, columns = history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 + datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_spider8_internal_wo_btest_APS' + str(p[8]) + '_' + str(p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[7]) + '_' + str(dd)
    
    model_grid.save( name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')
    
    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
    GINI = 2*(metrics.roc_auc_score(y_val , predict_class_val[:,1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score(y_test, predict_class_test[:,1])
    GINI_t = 2*(metrics.roc_auc_score(y_test, predict_class_test[:,1])) - 1
    
    result_all_8.at[j , 'name_model'] = name_m
    result_all_8.at[j ,'params'] = str(p)
    result_all_8.at[j ,'val_GINI'] = GINI
    result_all_8.at[j ,'val_APS'] = APS
    result_all_8.at[j ,'test_GINI'] = GINI_t
    result_all_8.at[j ,'test_APS'] = APS_t
    
    result_all_8.to_csv('internal_result_spider_8_wo_btest_APS.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t) 

124 (185400, 124) (False, 0.0002, 7, 5, 64, 0.25, 0.001, 0.0002, 20)
1 0.001
2 0.016
3 0.081
4 0.256
Instructions for updating:
Please use Model.fit, which supports generators.
train_begin | time:  16.05.2021 01:29:06
Epoch 1/150
epoch_end | time:  16.05.2021 01:50:17
 — val_aps:  0.059090 — val_a: 0.936557
2945/2945 - 1297s - loss: 0.0154 - accuracy: 0.9737 - auc: 0.9938 - precision: 0.9737 - recall: 0.9737 - val_loss: 0.1234 - val_accuracy: 0.9650 - val_auc: 0.9866 - val_precision: 0.9650 - val_recall: 0.9650
Epoch 2/150
epoch_end | time:  16.05.2021 02:11:37
 — val_aps:  0.050181 — val_a: 0.929911
2945/2945 - 1279s - loss: 0.0074 - accuracy: 0.9861 - auc: 0.9973 - precision: 0.9861 - recall: 0.9861 - val_loss: 0.0653 - val_accuracy: 0.9812 - val_auc: 0.9971 - val_precision: 0.9812 - val_recall: 0.9812
Epoch 3/150
epoch_end | time:  16.05.2021 02:33:03
 — val_aps:  0.071084 — val_a: 0.943879
2945/2945 - 1284s - loss: 0.0067 - accuracy: 0.9870 - auc: 0.9979 - precision: 0.9870 - recal

Epoch 27/150
epoch_end | time:  16.05.2021 11:04:41
 — val_aps:  0.067191 — val_a: 0.926290
2945/2945 - 1260s - loss: 0.0044 - accuracy: 0.9898 - auc: 0.9991 - precision: 0.9898 - recall: 0.9898 - val_loss: 0.0320 - val_accuracy: 0.9920 - val_auc: 0.9991 - val_precision: 0.9920 - val_recall: 0.9920
Epoch 28/150
epoch_end | time:  16.05.2021 11:25:40
 — val_aps:  0.054070 — val_a: 0.921566
2945/2945 - 1259s - loss: 0.0044 - accuracy: 0.9898 - auc: 0.9991 - precision: 0.9898 - recall: 0.9898 - val_loss: 0.0328 - val_accuracy: 0.9913 - val_auc: 0.9982 - val_precision: 0.9913 - val_recall: 0.9913
Epoch 29/150
epoch_end | time:  16.05.2021 11:46:41
 — val_aps:  0.042083 — val_a: 0.897536
2945/2945 - 1259s - loss: 0.0044 - accuracy: 0.9897 - auc: 0.9991 - precision: 0.9897 - recall: 0.9897 - val_loss: 0.0705 - val_accuracy: 0.9809 - val_auc: 0.9954 - val_precision: 0.9809 - val_recall: 0.9809
Epoch 30/150
epoch_end | time:  16.05.2021 12:07:47
 — val_aps:  0.057319 — val_a: 0.936081
2945/294

In [51]:
conda list

# packages in environment at C:\Anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
absl-py                   0.10.0                   pypi_0    pypi
aio-pika                  6.4.1                    pypi_0    pypi
aiormq                    3.2.3                    pypi_0    pypi
alabaster                 0.7.12                   py37_0  
alembic                   1.4.1                    pypi_0    pypi
anaconda                  2019.10                  py37_0  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.7                    py37_0  
anaconda-project          0.8.3                      py_0  
aniso8601                 8.0.0                    pypi_0    pypi
ansi2html                 1.5.2                    pypi_0    pypi
appdirs                   1.4.4                    pypi_0    pypi
argcomplete               1.10.0                   pypi_0    pypi


netcdf4                   1.5.4                    pypi_0    pypi
networkx                  2.3                        py_0  
nltk                      3.5                      pypi_0    pypi
noisereduce               1.1.0                    pypi_0    pypi
nose                      1.3.7                    py37_2  
notebook                  6.0.1                    py37_0  
numba                     0.45.1           py37hf9181ef_0  
numexpr                   2.7.0            py37hdce8814_0  
numpy                     1.18.5                   pypi_0    pypi
numpydoc                  0.9.1                      py_0  
nvgpu                     0.8.0                    pypi_0    pypi
oauthlib                  3.1.0                    pypi_0    pypi
olefile                   0.46                     py37_0  
opencv-contrib-python     4.5.1.48                 pypi_0    pypi
opencv-python             4.5.1.48                 pypi_0    pypi
openpyxl                  3.0.5                    p

wasabi                    0.8.2                    pypi_0    pypi
wcwidth                   0.1.7                    py37_0  
webencodings              0.5.1                    py37_1  
webob                     1.8.6                    pypi_0    pypi
websockets                8.1                      pypi_0    pypi
werkzeug                  0.16.0                     py_0  
wheel                     0.35.0                   pypi_0    pypi
widgetsnbextension        3.5.1                    py37_0  
win-unicode-console       0.5                      pypi_0    pypi
win_inet_pton             1.1.0                    py37_0  
wincertstore              0.2                      py37_0  
winpty                    0.4.3                         4  
woe                       0.1.4                    pypi_0    pypi
wordcloud                 1.8.1                    pypi_0    pypi
wrapt                     1.12.1                   pypi_0    pypi
xarray                    0.16.0                   p

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename C:\Anaconda3\conda-meta\packaging-19.2-py_0.json.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename C:\Anaconda3\conda-meta\wrapt-1.11.2-py37he774522_0.json.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename C:\Anaconda3\conda-meta\jupyterlab-1.1.4-pyhf63ae98_0.json.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename C:\Anaconda3\conda-meta\scikit-learn-0.21.3-py37h6288b17_0.json.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Coul


Note: you may need to restart the kernel to use updated packages.
